<a href="https://colab.research.google.com/github/aayushkubb/nlp/blob/main/Classification_using_Transfomers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
train=pd.read_csv('/content/drive/MyDrive/Quora/QuoratrainSet.csv')
test=pd.read_csv('/content/drive/MyDrive/Quora/Quoratestdata.csv')

# Zero Shot Models

In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [2]:
text = "The company's quarterly earnings increased by 20%, exceeding market expectations."
candidate_labels = ["finance", "sports", "politics", "technology"]

result = pipe(text, candidate_labels)

print(result)


{'sequence': "The company's quarterly earnings increased by 20%, exceeding market expectations.", 'labels': ['finance', 'technology', 'sports', 'politics'], 'scores': [0.6282318234443665, 0.22458064556121826, 0.08779595792293549, 0.05939159542322159]}


In [5]:
text = "The company's quarterly earnings increased by 20%, exceeding market expectations."
candidate_labels = ["money", "finance","wealth"]

result = pipe(text, candidate_labels)

print(result)


{'sequence': "The company's quarterly earnings increased by 20%, exceeding market expectations.", 'labels': ['money', 'finance', 'wealth'], 'scores': [0.6493109464645386, 0.2030172199010849, 0.14767183363437653]}


In [6]:
text = 'What can you say about feminism?'
candidate_labels = ["Sincere Question", "InSincere Question"]

result = pipe(text, candidate_labels)

print(result)


{'sequence': 'What can you say about feminism?', 'labels': ['Sincere Question', 'InSincere Question'], 'scores': [0.6316617131233215, 0.36833831667900085]}


In [12]:
text=train[train['target']==1].sample(1)['question_text'].tolist()[0]
print(text)

candidate_labels = ["Sincere Question", "InSincere Question"]

result = pipe(text, candidate_labels)

print(result)


Why a most people stupid and incapable of thinking for themselves? My IQ : 164-180
{'sequence': 'Why a most people stupid and incapable of thinking for themselves? My IQ : 164-180', 'labels': ['Sincere Question', 'InSincere Question'], 'scores': [0.5579420924186707, 0.44205790758132935]}


# Few shot Learning -1

In [13]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

Device set to use cuda:0


In [38]:
few_shot_examples = []
for label in [0, 1]:
    examples = train[train['target'] == label].sample(10)['question_text'].tolist()
    for example in examples:
        if label ==0:
          few_shot_examples.append({'text': example, 'label': "Sincere Question"})
        else:
          few_shot_examples.append({'text': example, 'label': "Insincere Question"})

In [39]:
few_shot_text = "\n".join([f"Example of {example['label']}: {example['text']}" for example in few_shot_examples])

few_shot_text

"Example of Sincere Question: How does a girl/boy feel while having first intercourse. Does it really pain hard?\nExample of Sincere Question: Is failure a stepping stone for success?\nExample of Sincere Question: Is change coordinator a good job?\nExample of Sincere Question: What is the difference between NATO standard and regular standard?\nExample of Sincere Question: Which are the top 10 careers in the world?\nExample of Sincere Question: Would you support an anonymous and random electoral system?\nExample of Sincere Question: Why is there confusion in America between a liberal capitalist government and a socialist government?\nExample of Sincere Question: Are there any contemporary celebrities or famous people who have more than two citizenships?\nExample of Sincere Question: How can you make someone apologize for what they did to you?\nExample of Sincere Question: How can I determine the chemical property of acetylene?\nExample of Insincere Question: What awaits an atheist after

In [36]:
# Dynamically construct the hypothesis template with few-shot examples
few_shot_text = "\n".join([f"Example of {example['label']}: {example['text']}" for example in few_shot_examples])
hypothesis_template = f"This example is a {{}}.\nFew-shot Examples:\n{few_shot_text}" # Added {} placeholder



In [41]:
text=train[train['target']==1].sample(1)['question_text'].tolist()[0]
candidate_labels = ["Sincere Question", "Insincere Question"]

# Use a correct hypothesis_template with a single placeholder for the candidate label
result = classifier(text, candidate_labels, hypothesis_template=hypothesis_template)


print(result)

{'sequence': 'Were there any Roman Catholic Popes that had a penchant for good looking young boys?', 'labels': ['Sincere Question', 'Insincere Question'], 'scores': [0.510753870010376, 0.489246129989624]}


# Few Shot Learning -2

In [44]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.8 MB/s eta 0:00:00


In [45]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
from datasets import Dataset

train_df=pd.read_csv('/content/drive/MyDrive/Quora/QuoratrainSet.csv')
test_df=pd.read_csv('/content/drive/MyDrive/Quora/Quoratestdata.csv')

In [43]:
model_name = "bert-base-uncased"  # Or another suitable model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # 2 labels: sincere/insincere

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [47]:
few_shot_examples = []
for label in [0, 1]:
    examples = train_df[train_df['target'] == label].sample(10)['question_text'].tolist()
    for example in examples:
        few_shot_examples.append({'text': example, 'label': label})

In [48]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_examples = [preprocess_function({'text': example['text']}) for example in few_shot_examples]

# Add labels to tokenized examples
for i, example in enumerate(tokenized_examples):
    example['labels'] = few_shot_examples[i]['label']

In [49]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,  # Adjust as needed
    num_train_epochs=3,  # Adjust as needed
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_examples,  # Use tokenized few-shot examples
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: aayushkubba-727 (aayushkubba-727-openai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss


TrainOutput(global_step=9, training_loss=0.5984752972920736, metrics={'train_runtime': 47.8947, 'train_samples_per_second': 1.253, 'train_steps_per_second': 0.188, 'total_flos': 15786663321600.0, 'train_loss': 0.5984752972920736, 'epoch': 3.0})

In [50]:
new_question = train_df[train_df['target']==0].sample(1)['question_text'].tolist()[0]
print(new_question)
inputs = tokenizer(new_question, return_tensors="pt")

# Move inputs to the same device as the model
device = next(model.parameters()).device  # Get model's device
inputs = inputs.to(device)  # Move inputs to the device

outputs = model(**inputs)

predicted_class = outputs.logits.argmax().item()  # 0 for sincere, 1 for insincere

print(f"Predicted class: {predicted_class}")

What is mpfi system?
Predicted class: 0


# Using LLMs

In [51]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `colab` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate wh

In [52]:
from transformers import pipeline

In [ ]:
generator = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.1")

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def classify_question(your_question):
    prompt = f"""
    Classify the following question as either "Sincere Question" or "Insincere Question".

    An insincere question is defined as a question intended to make a statement rather than look for helpful answers. Some characteristics that can signify that a question is insincere:

    -Has a non-neutral tone
    -Has an exaggerated tone to underscore a point about a group of people
    -Is rhetorical and meant to imply a statement about a group of people
    -Is disparaging or inflammatory
    -Suggests a discriminatory idea against a protected class of people, or seeks confirmation of a stereotype
    -Makes disparaging attacks/insults against a specific person or group of people
    -Based on an outlandish premise about a group of people
    -Disparages against a characteristic that is not fixable and not measurable
    -Isn't grounded in reality
    -Based on false information, or contains absurd assumptions
    -Uses sexual content (incest, bestiality, pedophilia) for shock value, and not to seek genuine answers

    ## Examples:

    **Sincere Question:** What are the best resources for learning Python?
    **Insincere Question:** Why are all politicians so incompetent?
    **Sincere Question:** How do I bake a chocolate cake?
    **Insincere Question:** Are all liberals snowflakes?

    ## Question to Classify:

    {your_question}
    """

    result = generator(prompt, max_new_tokens=10)
    prediction = result[0]['generated_text'].strip()
    return prediction

In [ ]:
classify_question("What are the best resources for learning Python?")

NameError: name 'generator' is not defined

In [ ]:
classify_question("Why are all politicians so incompetent?")